In [180]:
import numpy as np
import pandas as pd
from pulp import *
from IPython.display import Image, display_png

In [181]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)
t = t.T.drop([
    'C00001', # H2O
#     'C00003', # NAD
#     'C00004', # NADH
    'C00005', # NADPH
    'C00006', # NADP
    'C00009', # Phosphate
    'C00010', # CoA
#     'C00011', # CO2
    'C00033', # Acetate
    'C00080', # H+
    'C00138', # Reduced ferredoxin
    'C00139', # Oxidized ferredoxin
    'C15602',
    'C15603'
]).T

In [182]:
h = [
    'R00200',
    'R00268',
    'R00341',
    'R00342',
    'R00351',
    'R00361',
    'R00658',
    'R01015',
    'R01061',
    'R01070',
    'R01082',
    'R01196',
    'R01197',
    'R01325',
    'R01512',
    'R01518',
#     'R08549',
    'R01786',
    'R01899',
    'R01900',
    'R02164',
    'R02740',
    'R04779',
    'R10343'
]

In [183]:
p = t.T[h].T

In [184]:
for i in p:
    if sum(p[i] != 0) == 0:
        p=p.drop(i, axis=1)

In [185]:
# g  = []
# for i in p.columns:
#     if sum(p[i] != 0.0) == 1:
#         g.append(i)
# g

In [186]:
# p = p.drop(g, axis=1)

In [187]:
x = [LpVariable(i) for i in p.columns]

In [188]:
f = pd.DataFrame([i for i in p.columns])
# (f[f[0] == 'C00024'].index)[0]

In [189]:
m = LpProblem(sense=LpMaximize)
m += x[(f[f[0] == 'C00091'].index)[0]]

In [190]:
x[(f[f[0] == 'C00091'].index)[0]]

C00091

In [193]:
capacity_plus = 100
capacity_minus = -100

for i in p.index:
    m += np.dot(list(p.T[i]), x) <= capacity_plus
    m += np.dot(list(p.T[i]), x) >= capacity_minus
    
# ATP <-> ADP
m += x[(f[f[0] == 'C00002'].index)[0]] - x[(f[f[0] == 'C00008'].index)[0]] <= capacity_plus
m += x[(f[f[0] == 'C00002'].index)[0]] - x[(f[f[0] == 'C00008'].index)[0]] <= capacity_plus
    
# 2-Oxoglutarate <-> CO2
m += (
    x[(f[f[0] == 'C00026'].index)[0]] + 
#     x[(f[f[0] == 'C00010'].index)[0]] + 
    x[(f[f[0] == 'C00003'].index)[0]] 
    - x[(f[f[0] == 'C00091'].index)[0]] -
    x[(f[f[0] == 'C00011'].index)[0]] 
    - x[(f[f[0] == 'C00004'].index)[0]] 
#     x[(f[f[0] == 'C00080'].index)[0]] 
    <= capacity_plus
)

m += (
    x[(f[f[0] == 'C00026'].index)[0]] + 
#     x[(f[f[0] == 'C00010'].index)[0]] + 
    x[(f[f[0] == 'C00003'].index)[0]] 
    - x[(f[f[0] == 'C00091'].index)[0]] -
    x[(f[f[0] == 'C00011'].index)[0]] 
    - x[(f[f[0] == 'C00004'].index)[0]] 
#     x[(f[f[0] == 'C00080'].index)[0]] 
    >= capacity_minus
)    

m += x[(f[f[0] == 'C00267'].index)[0]] <= capacity_plus
m += x[(f[f[0] == 'C00011'].index)[0]] >= capacity_minus

In [203]:
status = m.solve()
print(LpStatus[status])

Optimal


In [206]:
value(m)

AttributeError: 'LpProblem' object has no attribute 'value'

AttributeError: 'LpProblem' object has no attribute 'value'